In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"

In [1]:
import sqlite3
from pathlib import Path

import pandas as pd
from docling.datamodel.base_models import PipelineOptions
from docling.datamodel.document import DocumentConversionInput
from docling.document_converter import DocumentConverter
from docling_core.types.doc.base import (
    BaseText,
    Ref,
    Table,
)

def format_raw_pdf(input_doc_paths,output_dir):
    doc_input = DocumentConversionInput.from_paths(input_doc_paths)

    pipeline_options = PipelineOptions()
    pipeline_options.do_ocr = True
    pipeline_options.do_table_structure = True
    pipeline_options.table_structure_options.do_cell_matching = True

    doc_converter = DocumentConverter(
        pipeline_options=pipeline_options
    )

    result = doc_converter.convert(doc_input)

    db_folder_path = output_dir / "table" 
    db_file_path = db_folder_path / "all_tables.db"
    db_folder_path.mkdir(parents=True, exist_ok=True)
    conn = sqlite3.connect(db_file_path)

    table_id = 0
    print(f'Extraction Starting ...')
    for r in result:
        
        output = r.output

        text_folder_path = output_dir / "text" 
        text_file_path = text_folder_path / f"{r.input.file.stem}.txt"
        text_folder_path.mkdir(parents=True, exist_ok=True)
        text_file_path.open("w").close()

        main_text_labels = [
            "paragraph",
            "table",
        ]

        has_title = False
        prev_text = ""
        if output.main_text is not None:
            for index,orig_item in enumerate(output.main_text):
                item = (
                    output._resolve_ref(orig_item)
                    if isinstance(orig_item, Ref)
                    else orig_item
                )
                if item is None:
                    continue

                item_type = item.obj_type
                if isinstance(item, BaseText) and item_type in main_text_labels:
                    text = item.text

                    # ignore repeated text
                    if prev_text == text:
                        continue
                    else:
                        prev_text = text

                    text_file_path.open("a").write(f"{text}\n\n")

                elif (
                    isinstance(item, Table)
                    and item.data
                    and item_type in main_text_labels
                ):
                    table = []
                    if(item.text == "Table of Contents"):
                        continue
                    for row in item.data:
                        tmp = []
                        for col in row:
                            tmp.append(col.text)
                        table.append(tmp)

                    if len(table) > 1 and len(table[0]) > 0:
                        table_df = pd.DataFrame(table[1:], columns=table[0])

                        c = conn.cursor()
                        table_name = ""
                        if(item.text!=""):
                            table_name = item.text
                        else:
                            table_name = f"table_{chr(ord('a')+table_id)}"
                            table_id+=1
                        table_df.to_sql(name=table_name, con=conn, if_exists='replace', index=False)
                        conn.commit()

    conn.close()
    print(f'Text and Tables are seperated from the PDF files and stored in the folder {output_dir}')

In [ ]:
input_doc_paths = [
    Path("./data/duplicate-chars.pdf")
]

output_dir = Path("./data/formatted/")

format_raw_pdf(input_doc_paths,output_dir)

### SQL Chain

Let's create a simple chain that takes a question, turns it into a SQL query, executes the query, and uses the result to answer the original question.

In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///./data/formatted/table/all_tables.db")

In [6]:
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import create_sql_query_chain

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [7]:
from langchain_core.prompts import PromptTemplate

sql_gen_prompt = PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template= 
"""You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
{table_info}

Question: {input}
SQLQuery: """)

In [8]:
write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

In [9]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
sql_chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

In [10]:
response = sql_chain.invoke({"question": "What is the Lower explosion limit of Hydrogen gas?"})
response

'The Lower explosion limit of Hydrogen gas is 4.0 vol%.'

### RAG Chain

We will create a RAG chain with text documents

In [11]:
from langchain_community.document_loaders import TextLoader

def get_text_documents(text_dir_path):
    if os.path.isdir(text_dir_path):
        text_documents = []
        for file in os.listdir(text_dir_path):
            loader = TextLoader(text_dir_path+file)
            text_documents.extend(loader.load())
    else:
        loader = TextLoader(text_dir_path)
        text_documents = loader.load()
    
    return text_documents

In [13]:
from langchain import hub
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

docs = get_text_documents('data/formatted/text/')
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/dccstor/kirushikesh/.conda/multimodalproject/lib/python3.10/site-packages/langsmith/client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [14]:
rag_chain.invoke("What is Riken Keiki GP-1000?")

'The Riken Keiki GP-1000 is a compact and lightweight gas detector with high sensitivity for hydrocarbons, using a catalytic sensor for measurement. It has a built-in pump with a pump booster function and allows direct selection from a list of 25 hydrocarbons for precise gas alignment. Calibration is only necessary for CH$_{4}$.'

### Multi-Modal Agent

In [17]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")

In [29]:
tools = [
    sql_chain.as_tool(
        name="sql_database",
        description="useful when you need to answer the user question from the tabular data. Use this for selective type questions.",
    ),
    rag_chain.as_tool(
        name="text_database",
        description="useful when you need to answer from text data corpus. It will retrieve the text and answer based on it. Use this for descriptive type questions.",
    ),
]

In [30]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)

In [31]:
agent_executor.invoke(
    {
        "input": "What is Riken Keiki GP-1000?"
    }
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `text_database` with `Riken Keiki GP-1000`


The Riken Keiki GP-1000 is a compact gas detector with high sensitivity for hydrocarbons, using a catalytic sensor for measurement. It has a built-in pump with a pump booster function and allows direct selection from a list of 25 hydrocarbons for precise gas alignment. Calibration is only necessary for CH$_{4}$.The Riken Keiki GP-1000 is a compact gas detector with high sensitivity for hydrocarbons. It uses a catalytic sensor for measurement and has a built-in pump with a pump booster function. It allows direct selection from a list of 25 hydrocarbons for precise gas alignment, and calibration is only necessary for CH$_{4}$.

> Finished chain.


{'input': 'What is Riken Keiki GP-1000?',
 'output': 'The Riken Keiki GP-1000 is a compact gas detector with high sensitivity for hydrocarbons. It uses a catalytic sensor for measurement and has a built-in pump with a pump booster function. It allows direct selection from a list of 25 hydrocarbons for precise gas alignment, and calibration is only necessary for CH$_{4}$.'}

In [34]:
agent_executor.invoke(
    {
        "input": "Identify the Lower explosion limit of Hydrogen gas?"
    }
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `sql_database` with `{'question': 'What is the Lower explosion limit of Hydrogen gas?'}`


The Lower explosion limit of Hydrogen gas is 4.0 vol%.The Lower explosion limit of Hydrogen gas is 4.0 vol%.

> Finished chain.


{'input': 'Identify the Lower explosion limit of Hydrogen gas?',
 'output': 'The Lower explosion limit of Hydrogen gas is 4.0 vol%.'}